In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/submission/submission.csv
/kaggle/input/g-research-crypto-forecasting/example_sample_submission.csv
/kaggle/input/g-research-crypto-forecasting/asset_details.csv
/kaggle/input/g-research-crypto-forecasting/example_test.csv
/kaggle/input/g-research-crypto-forecasting/train.csv
/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/__init__.py


In [2]:
import numpy as np
import pandas as pd
import gresearch_crypto
env = gresearch_crypto.make_env() #environment
#DF = pd.read_csv(r'train.csv')

In [3]:
DF = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', low_memory=False)

In [4]:
#DF2 = pd.read_csv(r'../input/submission/submission.csv')
#DF2.to_csv(r'./submission.csv')

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #Ignore errors

def __createLSTM_model(neurons=[1,1],LSTMneurons=2):
        model = Sequential() #NN Model
        model.add(Dense(neurons[0])) #First Dense layer
        model.add(LSTM(LSTMneurons)) #LSTM Layer
        model.add(Flatten()) #Flatten 2 D
        for n in neurons[1:]:
            model.add(Dense(n,activation='tanh')) #Trainable Layers
            model.add(Dropout(0.1))
        model.add(Dense(1, activation='linear')) #linear
        model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['MeanSquaredError'])	 #Regression
        return model

def createLSTM_model(neurons=[1,1],LSTMneurons=2,**kwargs):
    return KerasRegressor(__createLSTM_model,
    neurons = neurons,LSTMneurons = LSTMneurons) #Wrapper

In [6]:
model = createLSTM_model(neurons=[64,128,64],LSTMneurons=32)
DF = DF.dropna()
X = DF.iloc[:,:-1]
Y = DF.iloc[:,-1]
del DF
Y.name = 'TargetPrice'
model.fit(np.array(X).reshape(X.shape[0],X.shape[1],1),np.array(Y).reshape(Y.shape[0],1),
epochs=8,batch_size=256)

Epoch 1/8
91745/91745 [==============================] - 400s 4ms/step - loss: nan - mean_squared_error: nan
Epoch 2/8
91745/91745 [==============================] - 408s 4ms/step - loss: nan - mean_squared_error: nan
Epoch 3/8
91745/91745 [==============================] - 402s 4ms/step - loss: nan - mean_squared_error: nan
Epoch 4/8
91745/91745 [==============================] - 408s 4ms/step - loss: nan - mean_squared_error: nan
Epoch 5/8
91745/91745 [==============================] - 405s 4ms/step - loss: nan - mean_squared_error: nan
Epoch 6/8
91745/91745 [==============================] - 402s 4ms/step - loss: nan - mean_squared_error: nan
Epoch 7/8
91745/91745 [==============================] - 408s 4ms/step - loss: nan - mean_squared_error: nan
Epoch 8/8
91745/91745 [==============================] - 392s 4ms/step - loss: nan - mean_squared_error: nan


In [7]:
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['Target'] = model.predict(np.array(test_df).reshape(test_df.shape[0],test_df.shape[1],1))
    sample_prediction_df.to_csv(r'./submission.csv',index=False)
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
